In [39]:
import numpy as np
import scipy.stats as ss
import time 
from sympy import *
import xlrd
import math
from scipy.optimize import minimize
from matplotlib import pyplot as plt
#from scipy.misc import derivative

In [93]:
######## Black Scholes ###############################
r = 0.0235
def d1(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r + sigma**2 / 2.) * T)/(sigma * np.sqrt(T))
 
def d2(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r - sigma**2 / 2.) * T) / (sigma * np.sqrt(T))

def BlackScholes(ty,S0, K, r, sigma, T):
    if ty=="C":
        return S0 * ss.norm.cdf(d1(S0, K, r, sigma, T)) - K * np.exp(-r * T) * ss.norm.cdf(d2(S0, K, r, sigma, T))
    elif ty=='P':
        return K * np.exp(-r * T) * ss.norm.cdf(-d2(S0, K, r, sigma, T)) - S0 * ss.norm.cdf(-d1(S0, K, r, sigma, T))
#test:
print('test this function:')
print(BlackScholes('C',120,130,0.235,0.1,30))

test this function:
119.887236836


In [72]:
def BS_vector(ty,S0, K, r, sigma, T,i):
    for j in range(len(K)):
        if K[0] <= 0:
            shift(F,K)
        BlackScholes(ty,S0, K[j], r, sigma[j], T)

In [85]:
######## Black Scholes for Matrix ###############################
def BS_Matrix(ty,S0, K, r, sigma, T):
    for i in range(len(S0)):
        BS_vector(ty,S0[i],K[i],r,sigma[i],T[i],i)
#test:
type(BS_Matrix('C',F,K,0.0235,MKT,expiries))

/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


NoneType

In [86]:
######## Shift Function ###############################
def shift(F,K):
    shift = 0.001 - K[0]
    for j in range(len(K)):
        K[j] = K[j] + shift
        F = F + shift

In [87]:
######## Derivative Function ###############################
# Second-order central
def sec_der(f,x,h):
    return (f(x+h)-2*f(x)+f(x-h))/h**2

#def sec_der(f):
 #   def df(x, h):
  #      return ( f(x+h)-2*f(x)+f(x-h) )/h**2
   # return df

In [88]:
######## inputs and outputs #########################################
while True:
    try:
        file_input = xlrd.open_workbook('market_data.xlsx')     # load market data
    except:
        print('Input file is not in the directory!')
    break
Market_data = file_input.sheet_by_name('Swaptions data')        # file input forward rates

In [89]:
######## set swaptions characteristics ###############################

strike_spreads=[]
j=0
while True:
    try:
        strike_spreads.append(int(Market_data.cell(1,3+j).value))
        j = j+1
    except:
        break
num_strikes = len(strike_spreads)

expiries=[]
i=0
while True:
        try:
            expiries.append(Market_data.cell(2+i,1).value)
            i = i + 1
        except:
            break

tenors=[]
i=0
while True:
    try:
        tenors.append(Market_data.cell(2+i,0).value)
        i = i + 1
    except:
        break
        
# to create the ATM forward rates
F = []
i=0
while True:
    try:
        F.append(Market_data.cell(2+i,2).value)
        i = i+1
    except:
        break

# to create the strike grid
K = np.zeros((len(F),num_strikes))
for i in range(len(F)):
    for j in range(num_strikes):
        K[i][j] = F[i] + 0.0001*(strike_spreads[j])  

# to create market volatilities            
MKT = np.zeros((len(F),num_strikes))
for i in range(len(F)):
    for j in range(num_strikes):
        MKT[i][j] = Market_data.cell(2+i,3+j).value


# set starting parameters
starting_guess = np.array([0.001,0.5,0,0.001])
alpha = len(F)*[starting_guess[0]]
beta = len(F)*[starting_guess[1]]
rho = len(F)*[starting_guess[2]]
nu = len(F)*[starting_guess[3]]

In [90]:
def V_K(K_):
    return BS_Matrix('C',F, K_, r, MKT, expiries)
# test:
type(V_K(K))

/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


NoneType

In [91]:
#test
dV_K = sec_der(V_K,K,0.01)

/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in double_scalars
  after removing the cwd from sys.path.
/Users/mengyangliu/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in double_scalars
  import sys


TypeError: unsupported operand type(s) for *: 'int' and 'NoneType'